In [1]:
import numpy as np
import scipy
import combo
import os
import sys
import csv
import time
import copy
import urllib
import matplotlib.pyplot as plt
import math
import pandas as pd
from scipy.stats import rankdata
from mpl_toolkits.mplot3d import Axes3D
from scipy.stats import multivariate_normal
%matplotlib inline

In [2]:
sys.modules['copy']

<module 'copy' from 'C:\\Users\\0752617\\Anaconda3\\lib\\copy.py'>

In [3]:
#須更改路徑
os.environ['R_HOME'] = 'C:/Program Files/R/R-3.6.1'
os.environ['R_USER'] = 'C:/Users/0752617/Anaconda3/Lib/site-packages/rpy2'
import tzlocal
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from numpy.linalg import inv
import scipy.stats
from scipy.stats import norm
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar
from bokeh.palettes import Category10
import generate_domain as gd
from fractions import Fraction
output_notebook()

Loading BokehJS ...

In [4]:
##須更改路徑
importr('nloptr', lib_loc="C:/Users/0752617/Documents/R/win-library/3.6")
importr('MaxPro', lib_loc="C:/Users/0752617/Documents/R/win-library/3.6")
importr('numbers', lib_loc="C:/Users/0752617/Documents/R/win-library/3.6")

rpy2.robjects.packages.Package as a <module 'numbers'>

In [5]:
robjects.r('''
maxpro <- function(seed, list.name, run.number){
  set.seed(seed)
  factor_num <- length(list.name)
  print(class(list.name[[1]][2]))
  design <- MaxProLHD(n=run.number, p=factor_num)$Design
  for(i in 1:factor_num)
  {
      design[,i] <- design[,i]*(list.name[[i]][[2]]-list.name[[i]][[1]])+list.name[[i]][[1]]
  }
  return(design)
}
''')

R object with classes: ('function',) mapped to:
<SignatureTranslatedFunction - Python:0x00000191A0C96048 / R:0x00000191A16286B8>

In [6]:
def f(x, D):
    #total = 2*np.exp(-pow(sum(x[0:7,0])-1, 2)/0.1)+4*np.exp(-pow(sum(x[7:14,0])-4, 2)/0.1)\
    #        +1*np.exp(-pow(sum(x[14:21,0])-7, 2)/0.1)
    #total = 2*sum(x[0,0:7])+4*sum(x[0,7:14])+sum(x[0,14:21])
#     total = 0
#     for i in range(D):
#         total = total+4*np.exp(-pow(x[0,i],2))
#     total = 0
#     for j in range(D):
#         total = total - pow(sum(x[0,0:(j+1)]), 2)
#    total = 0
#     mu = np.array([1]).repeat(D)
#     Sigma = np.eye(D)
#     rv1 = multivariate_normal(2*mu, Sigma)
#     rv2 = multivariate_normal(3*mu, Sigma)
    #rv3 = multivariate_normal(2.5*mu, Sigma)
    #total = pow(2*np.pi, D/2)*rv1.pdf(x) + pow(2*np.pi, D/2)*rv2.pdf(x) + 4*pow(2*np.pi, D/2)*rv3.pdf(x)
            
    #total = sum(x[0, 0:14])
#     total = 2*np.exp(-(pow(x[:,0]-0.5, 2)+pow(x[:,1]-0.5, 2)+pow(x[:,2]-0.5, 2))/0.1)\
#             +4*np.exp(-(pow(x[:,0]-1.5, 2)+pow(x[:,1]-1.5, 2)+pow(x[:,2]-1.5, 2))/0.1)\
#             +1*np.exp(-(pow(x[:,0]-2.5, 2)+pow(x[:,1]-2.5, 2)+pow(x[:,2]-2.5, 2))/0.1)
    
#     total = np.exp(-(pow(x[0,0]-1, 2)+pow(x[0,1]-1, 2))/0.1)+4*np.exp(-(pow(x[0,0]-4, 2)+pow(x[0,1]-4, 2))/0.1)\
#             +2*np.exp(-(pow(x[0,0]-7, 2)+pow(x[0,1]-7, 2))/0.1)
    mu1 = np.linspace(-2, 0, D+1)[:-1]
    mu2 = np.linspace(0, 2, D+1)[:-1]
    mu3 = np.linspace(2, 4, D+1)[:-1]
    total = np.log(3*np.exp(-sum(pow(x[0]-mu1,2)))+np.exp(-sum(pow(x[0]-mu2,2)))+9*np.exp(-sum(pow(x[0]-mu3,2))))
    
    return total

def change_params(new, ori_params):
    min_len = []
    maxpro_params = []
    for i in range(len(ori_params)):
        min_len.append(min(abs(new[i]-ori_params[i][0]), abs(new[i]-ori_params[i][1])))
    min_value = min(min_len)
    for i in range(len(ori_params)):
        maxpro_params.append([new[i]-min_value, new[i]+min_value])
    return maxpro_params

def rand_search(params, n):
    rand_params = np.zeros((n, len(params)))
    for i in range(len(params)):
        rand_params[:,i] = np.random.uniform(params[i][0], params[i][1], n)
    return rand_params

In [7]:
    #X_range = [10, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 50000, 80000, 100000, 150000, 200000]
    #for search_points in X_range:
    for seed in np.arange(0, 5):
        true_value = []
        np.random.seed(seed)
        x = [-2, 4]
        D = 21
        ori_params = np.tile(x,(D,1)).tolist()
        initial_points = 10
        #training_X = np.array([np.loadtxt('Fixtime/4exp/d='+str(D)+'/init_point'+str(seed)+'.csv',delimiter=',')])
        training_X = robjects.r['maxpro'](int(seed), ori_params, initial_points)
        training_X = pandas2ri.ri2py(training_X)
        #np.savetxt('Fixtime/thirdmod/trainingseedrand'+str(seed)+'expx.csv', training_X, fmt="%.7f", delimiter=",")
        sigma_y = 5
        search_points = 5000
        t3 = time.time()
        X = rand_search(ori_params, search_points)
        #X = np.round(rand_search(ori_params, search_points))
        t4 = time.time()
        print(t4-t3)
        X = np.vstack((training_X, X))
        original = copy.copy(X)
        unadj_cols = None
        policy = combo.search.discrete.policy(test_X = original, test2_X = X, unadjust_cols = unadj_cols, initial = initial_points, search = search_points, fix_num=True)
        policy.set_seed( 0 )
        actions = np.arange(initial_points)
        policy.actions = policy.delete_actions(actions)
        print('Best', X[actions])
        t = []
        for i in range(len(actions)):
            value = f(training_X[[i]], D)+np.random.normal(0, sigma_y)
            t.append(value)
            true_value.append(f(training_X[[i]], D))
        t = np.array(t)
        print('Y_train', t)
        #np.savetxt('Fixtime/thirdmod/trainingseedrand'+str(seed)+'expy.csv', t, fmt="%.7f", delimiter=",")
        policy.write(actions, t)
        combo.search.utility.show_search_results(policy.history, len(actions))
        t1 = time.time()
        num_search_each_probe = 1 #how many points you want
        iter = 1
        num_rand_basis = 0

        while (iter <= 250): #condition need to change to different problem
        #while (t3-t1) <= 900:
        #while (t >= 3.95).any() != True:
            policy.iter = iter
            print(len(policy.test.X))
            actions = policy.bayes_search(max_num_probes = 1, num_search_each_probe = num_search_each_probe, simulator = None, score='EI', interval = 0,  num_rand_basis = num_rand_basis)
            print("features = ", original[actions])
            t = f(original[actions], D)+np.random.normal(0, sigma_y, len(original[actions]))
            t = np.array(t)
            true_value.append(f(original[actions], D))
            print('value =', t)
            policy.write(actions, t)
            combo.search.utility.show_search_results(policy.history, num_search_each_probe)  # describe search results
            ##update
            if iter%2==0 :
                maxpro_params = change_params(policy.training.X[-1,:], ori_params)
                rand_X = rand_search(maxpro_params, search_points)
                #rand_X = np.round(rand_search(maxpro_params, search_points))
            else:
                rand_X = rand_search(ori_params, search_points)
                for i in range(len(ori_params)):
                    rand_X[:,i] = (rand_X[:,i]+policy.training.X[-1,:][i])/2
                    #rand_X[:,i] = np.round((rand_X[:,i]+policy.training.X[-1,:][i])/2)
            if iter < 3:
                policy.test.X = np.vstack((policy.test.X, rand_X))
                original = copy.copy(policy.test.X)
                policy.actions = np.append(policy.actions, np.arange(initial_points+iter*search_points, initial_points+(iter+1)*search_points))
            else:
                policy.test.X = np.vstack((policy.test.X[policy.random_pt], rand_X))
                original = copy.copy(policy.test.X)
                policy.actions = np.arange(len(policy.test.X))
            iter = iter+1
            #seed = seed+10
            t3 = time.time()
            print('Chosen time =', t3-t1)
        res=policy.history
        best_fx, best_action = res.export_all_sequence_best_fx()
        path = 'Fixtime/thirdmod/FIXNDPS/d=21rand'+str(search_points)+'true.csv'
        if not os.path.isfile(path):
            np.savetxt(path, true_value, fmt="%.7f", delimiter=",")
        else:
            data = pd.read_csv(path, header=None)
            data = pd.concat([data, pd.DataFrame(true_value)], axis=1)
            data.to_csv(path, index=False, header=False)

0.015619993209838867
Best [[-1.7  3.7  1.9  0.1 -1.7  0.1  1.3  2.5  0.1 -1.1  0.7  0.1 -1.7  0.7
  -1.7  3.1  3.7  0.7  0.1  1.9  0.1]
 [-1.1  3.1  3.1  1.9  1.9  2.5 -1.1  0.7  2.5  0.7  0.1 -0.5  1.9 -1.1
  -1.1 -0.5 -0.5  2.5  3.1  3.7  3.7]
 [-0.5  2.5  0.1  3.7  2.5  3.1  1.9  3.7  1.9  3.7 -1.7  2.5  0.1  3.7
   2.5  1.3  0.7  1.3 -0.5 -1.1  0.7]
 [ 0.1 -1.1  3.7 -1.1 -0.5 -0.5 -0.5  0.1 -0.5  1.9  2.5  3.1 -0.5  2.5
   0.1  0.7  1.9  3.1 -1.7 -1.7  3.1]
 [ 0.7  1.3 -1.1  0.7  3.1 -1.1 -1.7  1.9 -1.7  2.5 -1.1 -1.7  2.5  0.1
   0.7  3.7  1.3 -1.7  1.3  0.1  1.9]
 [ 1.3  0.1  1.3 -1.7  0.7  1.9  3.1 -0.5  0.7  3.1  3.7 -1.1 -1.1 -0.5
   3.7  2.5  0.1  1.9  3.7  0.7 -1.1]
 [ 1.9 -1.7  0.7  2.5  0.1  3.7  2.5 -1.7 -1.1 -0.5  1.3  1.9  3.1 -1.7
  -0.5 -1.1  3.1 -0.5  1.9 -0.5  1.3]
 [ 2.5 -0.5 -0.5 -0.5  1.3  0.7  0.1  3.1  3.1 -1.7 -0.5  3.7  1.3  1.9
   3.1  0.1  2.5 -1.1  2.5  3.1 -1.7]
 [ 3.1  0.7  2.5  3.1  3.7 -1.7  3.7 -1.1  3.7  0.1  1.9  0.7  0.7  1.3
   1.3  1.9 -1.1  0.1 

0
choosing time= 0.01994490623474121
features =  [[-1.82255689 -0.46926901  1.62733424  0.32030637 -0.6805052   0.75651737
   0.40035547 -0.74594058  0.2264632   1.01047084 -1.50881797  0.06214647
  -1.35542437  1.66731122  3.81681072  2.41907128  3.05427872  3.76628441
  -0.02948203  1.41017862  3.56327601]]
value = [-40.25937094]
0016-th step: 
   current best f(x) = -40.259371 (best action=40) 

Chosen time = 4.9233410358428955
15000
Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0-th epoch, marginal likelihood 67.97826297865367
50-th epoch, marginal likelihood 66.37999917610665
100-th epoch, marginal likelihood 65.42546552851901
150-th epoch, marginal likelihood 64.85261579209342
200-th epoch, marginal likelihood 64.49354778720037
250-th epoch, marginal likelihood 64.25349942939539
300-th epoch, marginal likelihood 64.07839935998166
350-th epoch, marginal likelihood 63.937634918694755
400-th epoch, marginal likelihood 63.81441140331232


300-th epoch, marginal likelihood 89.1470740311037
350-th epoch, marginal likelihood 88.71485157276155
400-th epoch, marginal likelihood 88.34121610287187
450-th epoch, marginal likelihood 88.0116313361799
500-th epoch, marginal likelihood 87.71506491540148
Done

 Parameters of Gaussian kernel 
 
 width  =  [3.34706897]
 scale  =  12.330307111749326
 scale2 =  152.036473470056
 

test updating time= 1.3343439102172852
0
choosing time= 0.031244993209838867
features =  [[-1.72323815  1.0905872  -0.43316464 -0.40944783 -0.20994567  2.12752722
  -0.26945478 -0.44807381 -0.87571283  1.58562697 -0.83087532 -0.00904448
   0.38192513  1.99449284  2.13300407  2.10356263  2.24123827  1.85117606
   0.43100321  1.18180675  2.83462937]]
value = [-29.69599299]
0023-th step: 
   current best f(x) = -29.198640 (best action=10365) 

Chosen time = 13.577754735946655
15000
Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0-th epoch, marginal likelihood 96.23711

Done

Start the hyper parameter learning ...
0-th epoch, marginal likelihood 120.70353775662069
50-th epoch, marginal likelihood 120.30341445284485
100-th epoch, marginal likelihood 120.00867146922256
150-th epoch, marginal likelihood 119.74527347067354
200-th epoch, marginal likelihood 119.46727739618669
250-th epoch, marginal likelihood 119.16875503512097
300-th epoch, marginal likelihood 118.86350343571064
350-th epoch, marginal likelihood 118.5679056587783
400-th epoch, marginal likelihood 118.29315847215048
450-th epoch, marginal likelihood 118.04422113847141
500-th epoch, marginal likelihood 117.82189081864674
Done

 Parameters of Gaussian kernel 
 
 width  =  [2.98208513]
 scale  =  17.171960197529994
 scale2 =  294.87621702555435
 

test updating time= 1.719923734664917
0
choosing time= 0.034738779067993164
features =  [[-0.57173501  1.75188014  0.42020885 -0.21063659 -0.04278807  1.24916173
  -0.29891166 -1.11815386 -0.14537456  2.54861078  0.12356447  0.93573441
   1.1397264 

Done

Start the hyper parameter learning ...
0-th epoch, marginal likelihood 151.8507336506128
50-th epoch, marginal likelihood 151.39288944805003
100-th epoch, marginal likelihood 150.97281443084464
150-th epoch, marginal likelihood 150.58578333925342
200-th epoch, marginal likelihood 150.23139524809267
250-th epoch, marginal likelihood 149.9086715597356
300-th epoch, marginal likelihood 149.61595360545255
350-th epoch, marginal likelihood 149.35117812915306
400-th epoch, marginal likelihood 149.11209096531843
450-th epoch, marginal likelihood 148.896392278451
500-th epoch, marginal likelihood 148.7018306245365
Done

 Parameters of Gaussian kernel 
 
 width  =  [3.67181562]
 scale  =  21.924167396564165
 scale2 =  480.6691160325671
 

test updating time= 1.7175507545471191
0
choosing time= 0.03124380111694336
features =  [[ 0.01393396  0.94316824  0.65212021 -0.53502015  1.60880952  0.78786069
   0.45378465 -1.31267907 -1.31228262  2.53443804  0.69376267  0.33396042
   1.66991886  1.4

...

100-th epoch, marginal likelihood 966.6563087594038
150-th epoch, marginal likelihood 959.19999651375
200-th epoch, marginal likelihood 948.0698468061378
250-th epoch, marginal likelihood 935.3100756632416
300-th epoch, marginal likelihood 922.3814715145593
350-th epoch, marginal likelihood 910.2056676390245
400-th epoch, marginal likelihood 895.4344059350992
450-th epoch, marginal likelihood 881.4905170273842
500-th epoch, marginal likelihood 870.1809856108043
Done

 Parameters of Gaussian kernel 
 
 width  =  [3.93583726]
 scale  =  16.607170567284832
 scale2 =  275.7981142508916
 

test updating time= 14.570024251937866
0
choosing time= 0.2892735004425049
features =  [[-1.01636458 -0.44469133  0.01079258 -0.01745597  0.75109655 -0.1582777
   0.85621535  0.70199197  0.31089036  0.21839306  1.74389483  1.41125181
   1.28350427  1.17551346  1.87592854  0.95085296  1.29399867  1.82710417
   3.59523409  1.5462911   1.76853828]]
value = [-11.51973978]
0232-th step: 
   current best f(x) =

0
choosing time= 0.3351001739501953
features =  [[-1.29547851 -0.59657642 -0.48671471  0.36603796  0.98043482 -0.1155478
   1.31493368 -0.16140075 -0.63828013  1.4346429   0.86877954  0.78347285
   0.78504411  0.62292153  1.63853542  1.65018163  2.06027668  1.85938934
   3.595762    2.12888707  1.29671109]]
value = [-9.35422977]
0238-th step: 
   current best f(x) = 6.006960 (best action=38) 

Chosen time = 1801.8807892799377
15000
Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0-th epoch, marginal likelihood 1064.0666196735444
50-th epoch, marginal likelihood 1009.6872712814288
100-th epoch, marginal likelihood 994.4028322833717
150-th epoch, marginal likelihood 983.3924074021121
200-th epoch, marginal likelihood 973.2760852626088
250-th epoch, marginal likelihood 959.696623927674
300-th epoch, marginal likelihood 944.7566753699128
350-th epoch, marginal likelihood 927.9139005412943
400-th epoch, marginal likelihood 911.8369571078583
450-t

100-th epoch, marginal likelihood 1014.884473763278
150-th epoch, marginal likelihood 1002.3287405765398
200-th epoch, marginal likelihood 989.7765719138857
250-th epoch, marginal likelihood 977.2976654977881
300-th epoch, marginal likelihood 962.1323569926435
350-th epoch, marginal likelihood 945.8054657571611
400-th epoch, marginal likelihood 929.7753135487277
450-th epoch, marginal likelihood 915.895108104317
500-th epoch, marginal likelihood 905.47781866062
Done

 Parameters of Gaussian kernel 
 
 width  =  [4.12061611]
 scale  =  17.050181449339632
 scale2 =  290.7086874554053
 

test updating time= 15.433042287826538
0
choosing time= 0.30830907821655273
features =  [[-1.08139285 -0.47958727  0.17287782  0.17005138  0.77351353 -0.22885758
   0.73451992  0.69619028 -0.68731997  0.79982137  1.6551709   0.48392967
   0.29263774  1.96945293  2.10175173  1.15464376  1.72910161  1.71098144
   2.93140013  1.650505    1.67097709]]
value = [-16.81174435]
0245-th step: 
   current best f(x)

0
choosing time= 0.3476738929748535
features =  [[-0.77988176 -0.5394742  -0.27372666 -0.48048258  0.70553636  0.3518311
   0.8244471   0.24940631 -0.30117635  0.38455108  1.79041799  1.00093986
   1.29388672  1.26468522  1.62097311  1.52178995  1.32691022  2.2527537
   3.08269328  2.0775427   1.80879712]]
value = [-4.08388879]
0251-th step: 
   current best f(x) = 6.006960 (best action=38) 

Chosen time = 2002.0263073444366
15000
Start the initial hyper parameter searching ...
Done

Start the hyper parameter learning ...
0-th epoch, marginal likelihood 1083.3486645956355
50-th epoch, marginal likelihood 1048.4482367569601
100-th epoch, marginal likelihood 1036.791961752625
150-th epoch, marginal likelihood 1027.3105197275884
200-th epoch, marginal likelihood 1014.3788134280453
250-th epoch, marginal likelihood 1002.0637882314263
300-th epoch, marginal likelihood 985.1064182737443
350-th epoch, marginal likelihood 967.5818370770228
400-th epoch, marginal likelihood 950.1068683093514
45

50-th epoch, marginal likelihood 1062.1195442128594
100-th epoch, marginal likelihood 1048.57303788141
150-th epoch, marginal likelihood 1035.2884432417334
200-th epoch, marginal likelihood 1015.199920853696
250-th epoch, marginal likelihood 992.3479681992343
300-th epoch, marginal likelihood 969.8376896924589
350-th epoch, marginal likelihood 953.4882782562808
400-th epoch, marginal likelihood 943.388568122146
450-th epoch, marginal likelihood 938.0350785990638
500-th epoch, marginal likelihood 934.707861088756
Done

 Parameters of Gaussian kernel 
 
 width  =  [4.56553743]
 scale  =  18.078645409480533
 scale2 =  326.83741984173156
 

test updating time= 19.60302209854126
0
choosing time= 0.3458545207977295
features =  [[-0.89025904 -0.46822802 -0.64665328 -0.51723873  0.64141403 -0.31244717
   0.66735881  0.2199804   0.28980508  0.49292569  1.63841952  1.09343783
   1.4853202   1.29194093  1.7630106   1.20818376  1.5890465   2.25826995
   3.39925891  1.93092951  2.04357753]]
value =